In [1]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import random
from collections import defaultdict

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms (Resize, Normalize)
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalization for stability
])

# Load CelebA dataset
celeba_dataset = datasets.CelebA(root="./data", transform=transform, download=False)

# Dataset size
print(f"Total CelebA images: {len(celeba_dataset)}")

# Visualize an image
def show_image(img, title=""):
    img = img.permute(1, 2, 0).numpy() * 0.5 + 0.5  # Unnormalize
    plt.imshow(img)
    plt.title(title)
    plt.axis("off")
    plt.show()



Total CelebA images: 162770


In [ ]:
x = [1, 2, 3, 4, 5]
y = [2, 3, 5, 7, 11]

plt.plot(x, y)
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Sample Plot')
plt.show()


In [2]:
sample_img, sample_label = celeba_dataset[2]

img = sample_img.permute(1, 2, 0).numpy() * 0.5 + 0.5  # Unnormalize

In [ ]:
# Display sample image
plt.imshow(img)

In [ ]:
import os
import torch
from facenet_pytorch import MTCNN
from torchvision import datasets, transforms
from PIL import Image

# Initialize MTCNN face detector
mtcnn = MTCNN(image_size=160, margin=20)

# CelebA dataset paths
data_dir = "data/celeba/img_align_celeba"
identity_file = "celeba/identity_CelebA.txt"

# Load identity mapping
id_map = {}
with open(identity_file, "r") as f:
    for line in f:
        img_name, person_id = line.strip().split()
        id_map[img_name] = int(person_id)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

# Custom Dataset for CelebA
class CelebADataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, id_map, transform=None):
        self.root_dir = root_dir
        self.id_map = id_map
        self.transform = transform
        self.img_names = list(id_map.keys())

    def __getitem__(self, index):
        img_name = self.img_names[index]
        img_path = os.path.join(self.root_dir, img_name)
        label = self.id_map[img_name]

        img = Image.open(img_path).convert("RGB")

        # Face Detection & Cropping
        face = mtcnn(img)
        if face is None:
            return self.__getitem__((index + 1) % len(self.img_names))  # Skip bad images

        if self.transform:
            face = self.transform(face)

        return face, label

    def __len__(self):
        return len(self.img_names)

# Load dataset
dataset = CelebADataset(data_dir, id_map, transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Dataset Size: {len(dataset)}")

In [ ]:
import torchvision
print(torchvision.__version__)